In [61]:
from PIL import Image 
from IPython.display import display 
import random
import json
import os
import glob

In [74]:
PROJECT_NAME = 'PFP Test'
TOTAL_IMAGES = 100 # Number of random unique images we want to generate
METADATA_PATH = './metadata/'
IMAGES_PATH = './images/'
METADATA_FILE_NAME = 'all-traits.json'

In [80]:
# Clean up generated files

for dir in [IMAGES_PATH, METADATA_PATH]:
    files = glob.glob(dir + '/**/*', recursive=True)
    for f in files:
        try:
            os.remove(f)
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
    if not os.path.exists(dir):
        os.mkdir(dir)

In [64]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

background = ["Blue", "Orange", "Purple", "Red", "Yellow"] 
background_weights = [30, 40, 15, 5, 10]

circle = ["Blue", "Green", "Orange", "Red", "Yellow"] 
circle_weights = [30, 40, 15, 5, 10]

square = ["Blue", "Green", "Orange", "Red", "Yellow"] 
square_weights = [30, 40, 15, 5, 10]

# Dictionary variable for each trait. 
# Eech trait corresponds to its file name

background_files = {
    "Blue": "blue",
    "Orange": "orange",
    "Purple": "purple",
    "Red": "red",
    "Yellow": "yellow",
}

circle_files = {
    "Blue": "blue-circle",
    "Green": "green-circle",
    "Orange": "orange-circle",
    "Red": "red-circle",
    "Yellow": "yellow-circle"   
}

square_files = {
    "Blue": "blue-square",
    "Green": "green-square",
    "Orange": "orange-square",
    "Red": "red-square",
    "Yellow": "yellow-square"  
          
}

In [65]:
## Generate Traits

allImages = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    new_image ["Background"] = random.choices(background, background_weights)[0]
    new_image ["Circle"] = random.choices(circle, circle_weights)[0]
    new_image ["Square"] = random.choices(square, square_weights)[0]
    
    if new_image in allImages:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    allImages.append(new_trait_image)
    


In [66]:
# Returns true if all images are unique
def all_images_unique(allImages):
    seen = list()
    return not any(i in seen or seen.append(i) for i in allImages)

print("Are all images unique?", all_images_unique(allImages))

Are all images unique? True


In [67]:
# Add token Id to each image
i = 0
for item in allImages:
    item["tokenId"] = str(i)
    i = i + 1

In [68]:
print(allImages)

[{'Background': 'Blue', 'Circle': 'Blue', 'Square': 'Green', 'tokenId': '0'}, {'Background': 'Orange', 'Circle': 'Orange', 'Square': 'Blue', 'tokenId': '1'}, {'Background': 'Yellow', 'Circle': 'Red', 'Square': 'Green', 'tokenId': '2'}, {'Background': 'Orange', 'Circle': 'Blue', 'Square': 'Blue', 'tokenId': '3'}, {'Background': 'Blue', 'Circle': 'Green', 'Square': 'Blue', 'tokenId': '4'}, {'Background': 'Purple', 'Circle': 'Yellow', 'Square': 'Blue', 'tokenId': '5'}, {'Background': 'Orange', 'Circle': 'Blue', 'Square': 'Yellow', 'tokenId': '6'}, {'Background': 'Yellow', 'Circle': 'Yellow', 'Square': 'Blue', 'tokenId': '7'}, {'Background': 'Blue', 'Circle': 'Yellow', 'Square': 'Yellow', 'tokenId': '8'}, {'Background': 'Blue', 'Circle': 'Orange', 'Square': 'Green', 'tokenId': '9'}, {'Background': 'Red', 'Circle': 'Blue', 'Square': 'Blue', 'tokenId': '10'}, {'Background': 'Red', 'Circle': 'Green', 'Square': 'Orange', 'tokenId': '11'}, {'Background': 'Purple', 'Circle': 'Green', 'Square': '

In [69]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
circle_count = {}
for item in circle:
    circle_count[item] = 0

square_count = {}
for item in square:
    square_count[item] = 0

for image in allImages:
    background_count[image["Background"]] += 1
    circle_count[image["Circle"]] += 1
    square_count[image["Square"]] += 1
    
print(background_count)
print(circle_count)
print(square_count)

{'Blue': 23, 'Orange': 21, 'Purple': 21, 'Red': 16, 'Yellow': 19}
{'Blue': 24, 'Green': 25, 'Orange': 22, 'Red': 12, 'Yellow': 17}
{'Blue': 24, 'Green': 23, 'Orange': 19, 'Red': 15, 'Yellow': 19}


In [70]:
#### Generate Metadata for all Traits 
with open(METADATA_PATH + METADATA_FILE_NAME, 'w') as outfile:
    json.dump(allImages, outfile, indent=4)

In [71]:
  
    
#### Generate Images    
for item in allImages:

    im1 = Image.open(f'./trait-layers/backgrounds/{background_files[item["Background"]]}.jpg').convert('RGBA')
    im2 = Image.open(f'./trait-layers/circles/{circle_files[item["Circle"]]}.png').convert('RGBA')
    im3 = Image.open(f'./trait-layers/squares/{square_files[item["Square"]]}.png').convert('RGBA')

    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)

    #Convert to RGB
    rgb_im = com2.convert('RGB')
    file_name = item["tokenId"] + ".png"
    rgb_im.save(IMAGES_PATH + file_name)
    
    
    

In [79]:
#### Generate Metadata for each Image    

IMAGES_BASE_URI = 'https://gateway.pinata.cloud/ipfs/QmabF8y8GxuYEXrzj5w2CMbkeH2aSyojYaMSoA4UAAv3WL/'
# IMAGES_BASE_URI = 'ipfs://QmabF8y8GxuYEXrzj5w2CMbkeH2aSyojYaMSoA4UAAv3WL/'

f = open(METADATA_PATH + METADATA_FILE_NAME,) 
data = json.load(f)

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    tokenId = i['tokenId']
    tokenIdTemplate = '0' * len(str(TOTAL_IMAGES))
    nameId = tokenIdTemplate[:-len(tokenId)] + tokenId
    token = {
        "image": IMAGES_BASE_URI + tokenId + '.png',
        "name": PROJECT_NAME + ' ' + nameId,
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Circle", i["Circle"]))
    token["attributes"].append(getAttribute("Square", i["Square"]))

    with open(METADATA_PATH + str(tokenId), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()